In [1]:
import pandas as pd
import numpy as np

In [2]:
df_net= pd.read_csv('transformacion_netflix.csv', delimiter=',', encoding='utf-8')
df_dis= pd.read_csv('transformacion_disney.csv', delimiter=',', encoding='utf-8')
df_hulu= pd.read_csv('transformacion_hulu.csv', delimiter=',', encoding='utf-8')
df_amazon= pd.read_csv('transformacion_amazon.csv', delimiter=',', encoding='utf-8')
#df_ratings=pd.read_csv('transformacion_ratings.csv', delimiter=',', encoding='utf-8')


In [3]:
#creo un archivo gral
df_plataformagral = pd.concat([df_net,df_hulu,df_amazon,df_dis])

In [2]:
#lo guardo
df_plataformagral.to_csv('totalplataformas.csv', index=False)

NameError: name 'df_plataformagral' is not defined

In [3]:
df_plataformagral = pd.read_csv('totalplataformas.csv', delimiter=',', encoding='utf-8')

Película (sólo película, no serie, etc) con mayor duración según año, plataforma y tipo de duración. La función debe llamarse get_max_duration(year, platform, duration_type) y debe devolver sólo la cadena del nombre de la película.

In [138]:
clasificacion = df_plataformagral[ (df_plataformagral.type=='movie')
& (df_plataformagral.listed_in!='documentaries')]
clasificacion

,id,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,duration_int,duration_type
6,ns7,s7,movie,my little pony: a new generation,"robert cullen, josé luis ucha","vanessa hudgens, kimiko glenn, james marsden, ...",NaN,2021-09-24,2021,pg,91 min,children & family movies,equestria's divided. but a bright-eyed hero be...,91.0,min
7,ns8,s8,movie,sankofa,haile gerima,"kofi ghanaba, oyafunmike ogunlano, alexandra d...","united states, ghana, burkina faso, united kin...",2021-09-24,1993,tv-ma,125 min,"dramas, independent movies, international movies","on a photo shoot in ghana, an american model s...",125.0,min
9,ns10,s10,movie,the starling,theodore melfi,"melissa mccarthy, chris o'dowd, kevin kline, t...",united states,2021-09-24,2021,pg-13,104 min,"comedies, dramas",a woman adjusting to life after a loss contend...,104.0,min
12,ns13,s13,movie,je suis karl,christian schwochow,"luna wedler, jannis niewöhner, milan peschel, ...","germany, czech republic",2021-09-23,2021,tv-ma,127 min,"dramas, international movies",after most of her family is murdered in a terr...,127.0,min
13,ns14,s14,movie,confessions of an invisible girl,bruno garotti,"klara castanho, lucca picon, júlia gomes, marc...",NaN,2021-09-22,2021,tv-pg,91 min,"children & family movies, comedies",when the clever but socially-awkward tetê join...,91.0,min
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1445,ds1446,s1446,movie,x-men origins: wolverine,gavin hood,"hugh jackman, liev schreiber, danny huston, wi...","united states, united kingdom",NaN,2009,pg-13,108 min,"action-adventure, family, science fiction",wolverine unites with legendary x-men to fight...,108.0,min
1446,ds1447,s1447,movie,night at the museum: battle of the smithsonian,shawn levy,"ben stiller, amy adams, owen wilson, hank azar...","united states, canada",NaN,2009,pg,106 min,"action-adventure, comedy, family",larry daley returns to rescue some old friends...,106.0,min
1447,ds1448,s1448,movie,eddie the eagle,dexter fletcher,"tom costello, jo hartley, keith allen, dickon ...","united kingdom, germany, united states",NaN,2016,pg-13,107 min,"biographical, comedy, drama","true story of eddie edwards, a british ski-jum...",107.0,min
1448,ds1449,s1449,movie,bend it like beckham,gurinder chadha,"parminder nagra, keira knightley, jonathan rhy...","united kingdom, germany, united states",NaN,2003,pg-13,112 min,"buddy, comedy, coming of age",despite the wishes of their traditional famili...,112.0,min


array(['movie', 'tv show'], dtype=object)

In [4]:

def get_max_duration(year:int,platform:str,duration_type:str):
    
    #declaro parametros
    if platform == 'netflix':
        inicioId = 'n'
    elif platform == 'hulu':
        inicioId = 'h'
    elif platform == 'disney':
        inicioId = 'd'
    elif platform == 'amazon':
        inicioId = 'a'
    else:
        return 'no se encuentra plataforma, ingrear una de las opciones: netflix, hulu, disney o amazon'
    

    if duration_type == 'pelicula':
        durationtype= 'movie' 
    else:
        return 'ingresar parametro : pelicula' 

  
    # df_plataformagral=  pd.read_csv('totalplataformas.csv', delimiter=',', encoding='utf-8')

    filtro = df_plataformagral[ (df_plataformagral.type==durationtype) #filtro por peliculas
                            & (df_plataformagral.id.str.contains(inicioId)) #filtro por primer letra de plataforma
                            & (df_plataformagral.release_year==year)]  #filtro por el año

    max = filtro['duration_int'].max() #guardo el valor maximo
    filtro1 = filtro.query("duration_int==@max") #consulto la duracion de ese valor
    pelicula = filtro1['title'] #guardo lo que quiero que retorne
    if  not pelicula.empty: #si segun los datos ingresados el dataframe no esta vacio
        return {'pelicula': pelicula.iloc[0]}
    else:
        return 'Los datos ingresados no se encuentran en la base de datos'
    

In [5]:
get_max_duration(2021,'disney', 'pelicula')

{'pelicula': 'cruella'}

Cantidad de películas por plataforma con un puntaje mayor a XX en determinado año 

Cantidad de películas (sólo películas, no series, etc.) según plataforma, con un puntaje mayor a XX en determinado año. La función debe llamarse get_score_count(plataforma, anotado, año) y debe devolver un int, con el total de películas que cumplen con lo solicitado.

In [6]:
df_score= pd.read_csv('score_movies.csv', delimiter=',', encoding='utf-8')


In [315]:
nett= df_plataformagral[df_plataformagral.id.str.contains('n')]
nett

,id,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,duration_int,duration_type
0,ns1,s1,movie,dick johnson is dead,kirsten johnson,NaN,united states,2021-09-25,2020,pg-13,90 min,documentaries,"as her father nears the end of his life, filmm...",90.0,min
1,ns2,s2,tv show,blood & water,NaN,"ama qamata, khosi ngema, gail mabalane, thaban...",south africa,2021-09-24,2021,tv-ma,2 seasons,"international tv shows, tv dramas, tv mysteries","after crossing paths at a party, a cape town t...",2.0,season
2,ns3,s3,tv show,ganglands,julien leclercq,"sami bouajila, tracy gotoas, samuel jouy, nabi...",NaN,2021-09-24,2021,tv-ma,1 season,"crime tv shows, international tv shows, tv act...",to protect his family from a powerful drug lor...,1.0,season
3,ns4,s4,tv show,jailbirds new orleans,NaN,NaN,NaN,2021-09-24,2021,tv-ma,1 season,"docuseries, reality tv","feuds, flirtations and toilet talk go down amo...",1.0,season
4,ns5,s5,tv show,kota factory,NaN,"mayur more, jitendra kumar, ranjan raj, alam k...",india,2021-09-24,2021,tv-ma,2 seasons,"international tv shows, romantic tv shows, tv ...",in a city of coaching centers known to train i...,2.0,season
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8802,ns8803,s8803,movie,zodiac,david fincher,"mark ruffalo, jake gyllenhaal, robert downey j...",united states,2019-11-20,2007,r,158 min,"cult movies, dramas, thrillers","a political cartoonist, a crime reporter and a...",158.0,min
8803,ns8804,s8804,tv show,zombie dumb,NaN,NaN,NaN,2019-07-01,2018,tv-y7,2 seasons,"kids' tv, korean tv shows, tv comedies","while living alone in a spooky town, a young g...",2.0,season
8804,ns8805,s8805,movie,zombieland,ruben fleischer,"jesse eisenberg, woody harrelson, emma stone, ...",united states,2019-11-01,2009,r,88 min,"comedies, horror movies",looking to survive in a world taken over by zo...,88.0,min
8805,ns8806,s8806,movie,zoom,peter hewitt,"tim allen, courteney cox, chevy chase, kate ma...",united states,2020-01-11,2006,pg,88 min,"children & family movies, comedies","dragged from civilian life, a former superhero...",88.0,min


In [7]:
clasificacion = df_plataformagral[ (df_plataformagral.type=='movie')
& (df_plataformagral.id.str.contains('n'))]
clasificacion.id

0          ns1
6          ns7
7          ns8
9         ns10
12        ns13
         ...  
8801    ns8802
8802    ns8803
8804    ns8805
8805    ns8806
8806    ns8807
Name: id, Length: 6131, dtype: object

In [8]:
# df_ratings['fecha']= pd.to_datetime(df_ratings['fecha']) #necesito separar solo el año en nueva columna
# df_ratings['year']= df_ratings['fecha'].dt.year
# df_ratings.head(1)

,userId,rating,timestamp,movieId,fecha,year
0,1,1.0,1425941529,as680,2015-03-09 22:52:09,2015


In [90]:
# filtroanio_plataforma= df_ratings[(df_ratings.year==2021) 
#             & (df_ratings.movieId.str.contains('n'))] #filtro año y plataforma


In [10]:
filtroanio_plataforma

,userId,rating,timestamp,movieId,fecha,year
1,1,4.5,1425942435,ns2186,2015-03-09 23:07:15,2015
3,1,5.0,1425941546,ns3663,2015-03-09 22:52:26,2015
6,1,4.5,1425941300,ns8282,2015-03-09 22:48:20,2015
9,1,4.0,1425942228,ns1358,2015-03-09 23:03:48,2015
10,1,5.0,1425941434,ns7360,2015-03-09 22:50:34,2015
...,...,...,...,...,...,...
11024221,124377,5.0,1424259871,ns2877,2015-02-18 11:44:31,2015
11024222,124377,5.0,1424259817,ns4,2015-02-18 11:43:37,2015
11024225,124377,5.0,1424259888,ns8799,2015-02-18 11:44:48,2015
11024231,124377,4.5,1424259902,ns2002,2015-02-18 11:45:02,2015


In [91]:
promedio=filtroanio_plataforma.groupby(['movieId'])['rating'].mean()
#promedio agrupado por pelicula

In [22]:
promedio

movieId
ns1       3.593023
ns10      3.558140
ns100     3.758065
ns1000    3.426471
ns1001    3.592593
            ...   
ns995     3.527778
ns996     3.633333
ns997     3.742857
ns998     3.884615
ns999     3.700000
Name: rating, Length: 8807, dtype: float64

In [13]:
promedio[promedio<=1].count()

0

In [92]:
filtroanio_plataforma.movieId.isin(clasificacion.id).value_counts()

Series([], Name: movieId, dtype: int64)

In [93]:
df_confiltro=filtroanio_plataforma[filtroanio_plataforma.movieId.isin(clasificacion.id)]#filtro los registros de los id que necesito
promedio_nuevo = df_confiltro.groupby(['movieId'])['rating'].mean() #saco promedio agrupando por id
(promedio_nuevo>2).count()

0

In [94]:
promediofiltro =filtroanio_plataforma[filtroanio_plataforma.movieId.isin(clasificacion.id)].groupby(['movieId'])['rating'].mean()
#
promediofiltro[promediofiltro>4].count()

0

In [277]:
clasificacion.id.isin(consulta_score.movieId).value_counts()

True    398
Name: id, dtype: int64

In [293]:
clasificacion = df_plataformagral[ (df_plataformagral.type=='movie')#busco  los id solo de las peliculas
                                    & (df_plataformagral.id.str.contains('n')) #filtro por plataforma
                                     & (df_plataformagral.release_year==2015)] #filtro por año
consulta_score= df_score[(df_score.movieId.str.contains('n'))]#filtro por plataforma
df_peliculas=consulta_score[consulta_score.movieId.isin(clasificacion.id)]#me quedo solo con los id que estan en ambos df
total_peliculas= df_peliculas.score[df_peliculas.score > 1].count()#filtro por el puntaje mayor a xx del campo score y cuento los valores totales
total_peliculas

398

In [267]:
df_ratings.groupby(['movieId'])['rating'].mean()

movieId
as1       3.467131
as10      3.439571
as100     3.609302
as1000    3.556701
as1001    3.585288
            ...   
ns995     3.515625
ns996     3.626518
ns997     3.530526
ns998     3.582645
ns999     3.494781
Name: rating, Length: 22998, dtype: float64

In [268]:
df_score = pd.read_csv(r'score_movies.csv', delimiter=',', encoding='utf-8')

In [269]:
df_score

,movieId,score
0,as1,3.467131
1,as10,3.439571
2,as100,3.609302
3,as1000,3.556701
4,as1001,3.585288
...,...,...
22993,ns995,3.515625
22994,ns996,3.626518
22995,ns997,3.530526
22996,ns998,3.582645


In [8]:
def get_score_count(platform, scored, year):  

    import datetime

    #declaro parametros
    if platform == 'netflix':
        plataforma = 'n'
    elif platform == 'hulu':
        plataforma = 'h'
    elif platform == 'disney':
        plataforma = 'd'
    elif platform == 'amazon':
        plataforma = 'a'
    else:
        return 'ingresar una de las opciones: netflix, hulu, amazon o disney'
    
    
    clasificacion = df_plataformagral[ (df_plataformagral.type=='movie')#busco  solo las peliculas
                                        & (df_plataformagral.id.str.contains(plataforma)) #filtro por plataforma
                                        & (df_plataformagral.release_year==year)]  #filtro por el año
                                        

    consulta_score= df_score[(df_score.movieId.str.contains(plataforma))] #filtro por plataforma 
    df_peliculas=consulta_score[consulta_score.movieId.isin(clasificacion.id)]#me quedo solo con los id que estan en ambos df
    total_peliculas = df_peliculas.score[df_peliculas.score >scored].count()#filtro por el puntaje mayor a xx del campo score y cuento los valores totales


    return {'plataforma' : platform , 'cantidad': total_peliculas, 'anio' : year, 'score' : scored}

In [9]:
get_score_count('netflix', 1, 2015)

{'plataforma': 'netflix', 'cantidad': 398, 'anio': 2015, 'score': 1}

Cantidad de películas (sólo películas, no series, etc) según plataforma. La función debe llamarse get_count_platform(platform) y debe devolver un int, con el número total de películas de esa plataforma. Las plataformas deben llamarse amazon, netflix, hulu, disney.

In [10]:
def get_count_platform(platform): 

    #Declaro parametros
    if platform == 'netflix':
        plataforma = 'n'
    elif platform == 'hulu':
        plataforma= 'h'
    elif platform == 'disney':
        plataforma= 'd'
    elif platform == 'amazon':
        plataforma= 'a'
    else:
        return print('ingrese plataforma: amazon, disney, hulu o netflix')

   

    consulta_plataforma= df_plataformagral[(df_plataformagral.id.str.contains(plataforma))#filtro por plataforma
                                           & (df_plataformagral.type=='movie')]#filtro por peliculas

                                           
    conteo_movies= consulta_plataforma.title.count()#cuento
    return { 'plataforma': platform, 'peliculas': conteo_movies}

In [11]:
get_count_platform('netflix')

{'plataforma': 'netflix', 'peliculas': 6131}

Actor que más se repite según plataforma y año. La función debe llamarse get_actor(platform, year) y debe devolver sólo la cadena con el nombre del actor que más se repite según la plataforma y el año dado.

In [231]:
filtroplat = df_plataformagral[(df_plataformagral.id.str.contains('n'))
                                & (df_plataformagral.release_year==2015)] #filtro por año y plataforma
lista= filtroplat['cast']
newlista = [x for x in lista if pd.isnull(x) == False]
lista_cast=[]
for i in newlista: #itero para separar actores en diferentes str
    separador= i.split(sep=',')
    for j in separador: #itero para eliminar espacios en cada nuevo str y agregar a lista
        valores=j.strip()
        lista_cast.append(valores) #agrego valores a lista_cast
    df_lista_cast= pd.Series(lista_cast) #transformo a dataframe
df_lista_cast.value_counts().max()

5

In [12]:
def get_actor(platform, year): 

    #declaro parametros
    if platform == 'netflix':
        plataforma= 'n'
    elif platform == 'disney':
        plataforma= 'd'
    elif platform == 'hulu':
        plataforma= 'h'
    elif platform == 'amazon':
        plataforma= 'a'
    else:
        return 'ingresar plataforma valida: amazon, disney, hulu o netflix'


    filtroplat = df_plataformagral[(df_plataformagral.id.str.contains(plataforma))
                                   & (df_plataformagral.release_year==year)] #filtro por año y plataforma

    if filtroplat.empty: #si devuelve df vacio
        
        return 'el año no se encuentra en el dataset'
    else:
        lista= filtroplat['cast'] #guardo en nueva variable los actores por peliculas
        newlista = [x for x in lista if pd.isnull(x) == False]#elimino flotantes

        lista_cast=[]
        for i in newlista: #itero para separar actores en diferentes str
            separador= i.split(sep=',')
            for j in separador: #itero para eliminar espacios en cada nuevo str y agregar a lista
                    valores=j.strip()
                    lista_cast.append(valores) #agrego valores a lista_cast
        df_lista_cast= pd.Series(lista_cast) #transformo a dataframe
        return {'plataforma': platform, 'anio': year, 'actor' : df_lista_cast.value_counts().idxmax(), 'apariciones' : df_lista_cast.value_counts().max()} #imprimo copnsulta del str mas repetido

In [13]:
get_actor('netflix', 2015)

{'plataforma': 'netflix',
 'anio': 2015,
 'actor': 'james franco',
 'apariciones': 5}

La cantidad de contenidos/productos (todo lo disponible en streaming) que se publicó por país y año. La función debe llamarse prod_per_county(tipo,pais,anio) deberia devolver la cantidada de contenidos/productos segun el tipo de contenido (pelicula,serie) por pais y año en un diccionario con las variables llamadas 'pais' (nombre del pais), 'anio' (año), 'pelicula' (cantidad de contenidos/productos).

In [33]:
pais= 'canada'


In [37]:
prueba.shape



(38, 15)

In [35]:
prueba= filtro_anioytipo[filtro_anioytipo.country.str.contains(pais)]
prueba.type.count()

38

In [30]:
filtro_anioytipo=df_plataformagral[(df_plataformagral.type=='movie') 
                                    & (df_plataformagral.release_year==2015)
                                    & (df_plataformagral.country.notnull())]


In [38]:
df_plataformagral.isna().sum()

id                   0
show_id              0
type                 0
title                0
director          8259
cast              5321
country          11499
date_added       10982
release_year         0
rating               0
duration           482
listed_in            0
description          4
duration_int       482
duration_type      482
dtype: int64

In [54]:
def prod_per_county(tipo,pais,anio):

    #declaramos parametros
    
    if tipo == 'peliculas' :
        variable_tipo= 'movie'
    elif tipo == 'series':
        variable_tipo= 'tv show'
    else:
        return 'ingresar parametro : peliculas o series'

    filtro=df_plataformagral[(df_plataformagral.type=='movie') 
                                    & (df_plataformagral.release_year==anio)
                                    & (df_plataformagral.country.notnull())
                                    & (df_plataformagral.country.str.contains(pais))]
    # total=filtro.type.count()
    #     prueba= filtro_anioytipo[filtro_anioytipo.country.str.contains(pais)]
    # prueba.type.count()
    respuesta= filtro.type.count()

    return {'pais': pais, 'anio': anio, variable_tipo: respuesta}

In [68]:
prod_per_county('peliculas', 'canada', 2020)

{'pais': 'canada', 'anio': 2020, 'movie': 30}

La cantidad total de contenidos/productos (todo lo disponible en streaming, series, peliculas, etc) según el rating de audiencia dado (para que publico fue clasificada la pelicula). La función debe llamarse get_contents(rating) y debe devolver el numero total de contenido con ese rating de audiencias.

In [72]:
df_plataformagral.rating.unique()

array(['pg-13', 'tv-ma', 'pg', 'tv-14', 'tv-pg', 'tv-y', 'tv-y7', 'r',
       'tv-g', 'g', 'nc-17', '74 min', '84 min', '66 min', 'nr',
       'tv-y7-fv', 'ur', 'not rated', '2 seasons', '93 min', '4 seasons',
       '136 min', '91 min', '85 min', '98 min', '89 min', '94 min',
       '86 min', '3 seasons', '121 min', '88 min', '101 min', '1 season',
       '83 min', '100 min', '95 min', '92 min', '96 min', '109 min',
       '99 min', '75 min', '87 min', '67 min', '104 min', '107 min',
       '103 min', '105 min', '119 min', '114 min', '82 min', '90 min',
       '130 min', '110 min', '80 min', '6 seasons', '97 min', '111 min',
       '81 min', '49 min', '45 min', '41 min', '73 min', '40 min',
       '36 min', '39 min', '34 min', '47 min', '65 min', '37 min',
       '78 min', '102 min', '129 min', '115 min', '112 min', '61 min',
       '106 min', '76 min', '77 min', '79 min', '157 min', '28 min',
       '64 min', '7 min', '5 min', '6 min', '127 min', '142 min',
       '108 min', '57 min'

In [ ]:
def get_contents(rating):
    